In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
print(tf.__version__)
prime1=[]
# import pandas as pd 
def SieveOfEratosthenes(n): 
       
    # Create a boolean array "prime[0..n]" and  
    # initialize all entries it as true. A value  
    # in prime[i] will finally be false if i is 
    # Not a prime, else true. 
    prime = [True for i in range(n+1)] 
      
    p = 2
    while(p * p <= n): 
           
        # If prime[p] is not changed, then it is  
       # a prime 
        if (prime[p] == True): 
               
            # Update all multiples of p 
            for i in range(p * p, n + 1, p): 
                prime[i] = False
        p += 1
    c = 0
  
    # Print all prime numbers 
    for p in range(2, n): 
        if prime[p]: 
            prime1.append(p)
    return prime1
  
prime=SieveOfEratosthenes(40000)

num=[]
newlist=[]
for i in range(len(prime)):
    s=prime[:i]
    newlist.append(sum(s))
    num.append(i)
 
 
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
dataset = pd.DataFrame(list(zip(num, prime1)), 
               columns =['num', 'sop']) 

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_dataset.describe().transpose()


2.3.1


,count,mean,std,min,25%,50%,75%,max
num,3362.0,2113.099941,1219.826583,1.0,1054.25,2123.5,3170.75,4202.0
sop,3362.0,18959.858418,11845.640866,3.0,8434.00,18547.0,29177.50,39989.0


In [13]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('sop')
test_labels = test_features.pop('sop')

In [14]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())


TypeError: 'NoneType' object is not subscriptable

In [100]:
horsepower = np.array(train_features['num'])

horsepower_normalizer = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer.adapt(horsepower)

In [101]:
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

horsepower_model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_19 (Normalizat (None, 1)                 3         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 2         
Total params: 5
Trainable params: 2
Non-trainable params: 3
_________________________________________________________________


In [102]:
horsepower_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [103]:
%%time
history = horsepower_model.fit(
    train_features['num'], train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Wall time: 4.52 s


In [104]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(100, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.0001))
  return model
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)
dnn_horsepower_model.summary()

dnn_horsepower_model.fit(
    train_features['num'], train_labels,
    validation_split=0.2,
    verbose=1, epochs=250)


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_19 (Normalizat (None, 1)                 3         
_________________________________________________________________
dense_37 (Dense)             (None, 100)               200       
_________________________________________________________________
dense_38 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 101       
Total params: 10,404
Trainable params: 10,401
Non-trainable params: 3
_________________________________________________________________
Epoch 1/250
11/11 [==============================] - 0s 11ms/step - loss: 1843.4464 - val_loss: 1825.8026
Epoch 2/250
11/11 [==============================] - 0s 3ms/step - loss: 1843.3993 - val_loss: 1825.7574
Epoch 3/250
11/11 [=================

In [105]:
print(len(prime))

550


In [106]:
a=[1,]
dnn_horsepower_model.predict(a)

array([[418.22]], dtype=float32)